# 使用clinicalBert将text嵌入成向量

In [20]:
import os
import torch
import csv
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel

# 加载模型和分词器

In [21]:
# 加载模型和 tokenizer
model_path = "../BERT_model/clinicalbert"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [22]:
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModel.from_pretrained(model_path).to(device)
model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(28996, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

# 读取文本，并且推理为向量，缓存到本地

In [23]:
def process_test_listfile(
    test_listfile_path: str,
    mode: str = "test",
    save_dir: str = "./embedding",
):
    os.makedirs(save_dir, exist_ok=True)

    # 读取列表文件内容（不使用 pandas）
    with open(test_listfile_path, newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        rows = list(reader)

    # 提取 text 内容
    def extract_text_for_row(note, stay):
        subject_id = note.split('_')[0]
        stay_id = stay.split('.')[0].split('_')[-1]
        note_path = f"../data/{mode}/{subject_id}/{note}"

        text = dict()
        try:
            with open(note_path, "r") as ntfile:
                current_doc_lines = []
                head = True
                for line in ntfile:


                    line = line.strip()
                    if line:
                        if head:
                            id_doc, doc_cat, chart_date, chart_time, h, icu_id, h_id = line.split(',')
                            # 只读取同一住院病人的笔记
                            if stay_id == icu_id:
                                id_doc = int(id_doc)
                                current_doc_lines = []
                                head = False
                            else:
                                head = False  # skip this block
                        else:
                            if stay_id == icu_id:
                                current_doc_lines.append(line)
                    else:
                        if stay_id == icu_id:
                            # 将整段 token 拼接成完整字符串，用空格连接
                            text[id_doc] = ' '.join(current_doc_lines)
                            current_doc_lines = []
                            head = True
                        else:
                            head = True
        except FileNotFoundError:
            print(f"[Missing note file] {note_path}")

        texts = list(text.values())
        return ' '.join(texts)

    def sliding_window_encode(text, max_len=512, stride=236):
        tokens = tokenizer.tokenize(text)
        input_ids_list = []
        attention_mask_list = []

        for i in range(0, len(tokens), stride):
            chunk = tokens[i:i + max_len - 2]
            if not chunk:
                break
            tokens_chunk = [tokenizer.cls_token] + chunk + [tokenizer.sep_token]
            ids = tokenizer.convert_tokens_to_ids(tokens_chunk)
            attention = [1] * len(ids)

            pad_len = max_len - len(ids)
            ids += [tokenizer.pad_token_id] * pad_len
            attention += [0] * pad_len

            input_ids_list.append(torch.tensor(ids))
            attention_mask_list.append(torch.tensor(attention))
        return input_ids_list, attention_mask_list

    def mean_pooling(last_hidden_state, attention_mask):
        mask = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * mask, 1)
        sum_mask = torch.clamp(mask.sum(1), min=1e-9)
        return sum_embeddings / sum_mask

    # 主处理循环
    for row in tqdm(rows):
        note = row['notes']
        stay = row['stay']
        label = row.get('label', None)  # 可选
        text = extract_text_for_row(note, stay)

        subject_id = note.split('_')[0]
        stay_id = stay.split('.')[0].split('_')[-1]
        filename = f"{subject_id}_{stay_id}.pt"
        save_path = os.path.join(save_dir, filename)

        try:
            input_ids_list, attention_mask_list = sliding_window_encode(text)

            if not input_ids_list:
                print(f"[Empty input] subject_id: {subject_id}, stay_id: {stay_id}")
                final_embedding = torch.zeros(768)
                torch.save(final_embedding, save_path)
                continue

            all_embeddings = []
            for input_ids, attention_mask in zip(input_ids_list, attention_mask_list):
                input_ids = input_ids.unsqueeze(0).to(device)
                attention_mask = attention_mask.unsqueeze(0).to(device)

                with torch.no_grad():
                    output = model(input_ids=input_ids, attention_mask=attention_mask)
                    pooled = mean_pooling(output.last_hidden_state, attention_mask)
                    all_embeddings.append(pooled.cpu())

            if not all_embeddings:
                print(f"[No embeddings] subject_id: {subject_id}, stay_id: {stay_id}")
                final_embedding = torch.zeros(768)
            else:
                final_embedding = torch.mean(torch.stack(all_embeddings), dim=0)

            torch.save(final_embedding, save_path)

        except Exception as e:
            print(f"[Error] subject_id: {subject_id}, stay_id: {stay_id}, error: {str(e)}")
            final_embedding = torch.zeros(768)
            torch.save(final_embedding, save_path)

In [24]:
process_test_listfile(
    test_listfile_path="../data/train/train_listfile.csv",
    mode="train",
    save_dir="./embedding"
)


  1%|          | 327/37150 [00:16<19:19, 31.75it/s] 

[Empty input] subject_id: 406, stay_id: 261109


  1%|          | 390/37150 [00:18<18:30, 33.11it/s]

[Empty input] subject_id: 494, stay_id: 268296


  2%|▏         | 739/37150 [00:38<23:23, 25.94it/s]  

...

 99%|█████████▉| 36768/37150 [1:02:39<00:55,  6.89it/s]

[Empty input] subject_id: 98249, stay_id: 277307


100%|█████████▉| 37088/37150 [1:03:24<00:04, 14.29it/s]

[Empty input] subject_id: 99756, stay_id: 288659


100%|██████████| 37150/37150 [1:03:32<00:00,  9.74it/s]


In [25]:
process_test_listfile(
    test_listfile_path="../data/train/val_listfile.csv",
    mode="train",
    save_dir="./embedding"
)


  2%|▏         | 86/4656 [00:07<06:40, 11.40it/s]

[Empty input] subject_id: 940, stay_id: 250778


  6%|▌         | 272/4656 [00:19<03:11, 22.94it/s]

[Empty input] subject_id: 3482, stay_id: 253555


  7%|▋         | 318/4656 [00:25<17:37,  4.10it/s]

...

 98%|█████████▊| 4552/4656 [07:41<00:16,  6.16it/s]

[Empty input] subject_id: 96240, stay_id: 221665


 98%|█████████▊| 4579/4656 [07:42<00:04, 18.71it/s]

[Empty input] subject_id: 97042, stay_id: 299545


100%|██████████| 4656/4656 [07:54<00:00,  9.80it/s]


In [26]:
process_test_listfile(
    test_listfile_path="../data/test/test_listfile.csv",
    mode="test",
    save_dir="./embedding"
)


  2%|▏         | 74/4689 [00:04<03:43, 20.64it/s]

[Empty input] subject_id: 975, stay_id: 243103


  6%|▌         | 263/4689 [00:22<04:10, 17.65it/s]

[Empty input] subject_id: 2639, stay_id: 278380


  9%|▉         | 432/4689 [00:38<06:39, 10.65it/s]

...

 92%|█████████▏| 4309/4689 [07:03<00:23, 16.22it/s]

[Empty input] subject_id: 85441, stay_id: 230447


 95%|█████████▍| 4450/4689 [07:21<00:14, 16.49it/s]

[Empty input] subject_id: 90317, stay_id: 228081
[Empty input] subject_id: 90317, stay_id: 206771


100%|██████████| 4689/4689 [07:54<00:00,  9.88it/s]
